In [ ]:
import nltk
nltk.download("stopwords")
from preprocessing import stem_, stem, create_dataset
import numpy as np
import joblib
import random
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
from distances import Cosine_measure_to_chain_av, Euclidian_measure_to_chain_av,  Manhattan_measure_to_chain_av

In [ ]:
# multilingial embedings dictionary
new_multi_dict = joblib.load('letter_mult_emb_2.pkl') 
'''
файл лежит в папке 
https://drive.google.com/drive/folders/1kIJmDcmSLBql-f3mh_ASfswPWSdMkaPj?usp=sharing 
'''

# thresholds dict for 3 distance metrics
thresholds = joblib.load('metric_experiment_result_load_lang.pkl')
# metric_experiment_result_load_lang.pkl для only load_lang
# metric_experiment_result.pkl для load_lang + labeling

# корпус из слуайных предложений словаря Ангелины
cluster = joblib.load('angels_cluster_load_lang.pkl')
# angels_cluster_load_lang.pkl для only load_lang
# angels_cluster.pkl для load_lang + labeling


def get_word(w):  
    # function for getting words from dictionary
    try:  
        for line in new_multi_dict[w[0].lower()][len(w)]:
            vec = line.split()
            word = vec[0]
            if w == word:
                vector = list(map(lambda x: float(x), vec[1:]))
                return vector
    except:
        return None

def make_preds(sentence):
    # predictions for sentence
    new_sentence = list(map(lambda x: get_word(x.lower()), sentence))
    new_sentence = np.array([x for x in new_sentence if x is not None]).mean(axis=0)

    preds_cos = Cosine_measure_to_chain_av(cluster, new_sentence) <= thresholds['cos'][0]
    preds_euc = Euclidian_measure_to_chain_av(cluster, new_sentence) <= thresholds['euc'][0]
    preds_manh = Manhattan_measure_to_chain_av(cluster, new_sentence) <= thresholds['manh'][0]

    return {'cos': int(preds_cos), 'euc': int(preds_euc), 'manh': int(preds_manh)}

